<a href="https://colab.research.google.com/github/TianyiRnj/dino-classification-extent/blob/main/PTDQ_with_dinov2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision pandas numpy matplotlib tqdm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00


In [2]:
!pip install torchinfo --quiet

In [3]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00


In [4]:
import os
import zipfile
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from PIL import Image

In [5]:
# Uncomment this if you want to use Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/11785/DL_project

Mounted at /content/gdrive
/content/gdrive/MyDrive/11785/DL_project


In [6]:
#!/bin/bash
!kaggle datasets download andrewmvd/isic-2019
!unzip -q isic-2019.zip -d '/content/sample_data'

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/isic-2019
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
isic-2019.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
%%writefile config.yaml

structure: 'normclassi'
num_epochs: 20
wandb: False

Overwriting config.yaml


In [8]:
import yaml
with open("config.yaml") as file:
    config = yaml.safe_load(file)
print(config)

{'structure': 'normclassi', 'num_epochs': 20, 'wandb': False}


CHECKPOINT

In [9]:
RESUME_TRAINING = True
PRE_CHECKPOINTS_PATH = f'/content/gdrive/MyDrive/11785/DL_project/checkpoints-basic/checkpoint-best-model.pth'
CHECKPOINTS_ROOT = f"/content/gdrive/MyDrive/11785/DL_project/checkpoints-PTQ"
os.makedirs(CHECKPOINTS_ROOT, exist_ok=True)
DATA_ROOT = '/content/sample_data'

ORGANIZE DATA

In [10]:
# --- Step 2: Organize Data ---
# Load metadata
train_meta = pd.read_csv(f"{DATA_ROOT}/ISIC_2019_Training_GroundTruth.csv")
train_meta['image'] = train_meta['image'] + '.jpg'

# Convert one-hot encoding to class labels
classes = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC']
train_meta['label'] = train_meta[classes].idxmax(axis=1)
label_to_idx = {cls: idx for idx, cls in enumerate(classes)}


In [11]:
train_df, temp_df = train_test_split(train_meta, test_size=0.3,
                                    stratify=train_meta['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5,
                                  stratify=temp_df['label'], random_state=42)

# --- Step 3: Define Dataset Class ---
class ISIC2019Dataset(Dataset):
    def __init__(self, df, img_dir=f'{DATA_ROOT}/ISIC_2019_Training_Input/ISIC_2019_Training_Input', transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['image'])
        image = Image.open(img_path).convert('RGB')
        label = self.label_to_idx[self.df.iloc[idx]['label']]

        if self.transform:
            image = self.transform(image)

        return image, label

In [12]:
# --- Step 4: Data Transforms and Loaders ---
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and loaders
batch_size = 64
train_dataset = ISIC2019Dataset(train_df, transform=train_transform)
val_dataset = ISIC2019Dataset(val_df, transform=val_transform)
test_dataset = ISIC2019Dataset(test_df, transform=val_transform)

workers = os.cpu_count()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=workers)

In [13]:
# --- Step 6: Model Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- Step 5: Handle Class Imbalance ---
train_labels = [label for _, label in train_dataset]
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

Using device: cpu


In [14]:
import torchvision.models as models
from torchsummary import summary

class DINOv2Classifier(nn.Module):
    def __init__(self, num_classes=8, pretrain_choice='frozen'):
        super().__init__()
        self.dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

        self.classifier = nn.Sequential(
            nn.Linear(self.dinov2.embed_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        # Get all image features (batch_size, num_patches, embed_dim)
        features = self.dinov2(x)
        features = self.dinov2.norm(features)

        # Handle different output formats:
        if features.dim() == 3:  # Standard case with spatial dimensions
            cls_token = features[:, 0, :]  # Extract [CLS] token

        else:  # Fallback for 2D output
            cls_token = features

        out = self.classifier(cls_token)
        return out

# model = DINOv2Classifier().to(device)
model = DINOv2Classifier(pretrain_choice='frozen').to(device)
summary(model, input_size=(3, 224, 224))

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 111MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 384, 16, 16]         226,176
          Identity-2             [-1, 256, 384]               0
        PatchEmbed-3             [-1, 256, 384]               0
         LayerNorm-4             [-1, 257, 384]             768
            Linear-5            [-1, 257, 1152]         443,520
           Dropout-6          [-1, 6, 257, 257]               0
            Linear-7             [-1, 257, 384]         147,840
           Dropout-8             [-1, 257, 384]               0
   MemEffAttention-9             [-1, 257, 384]               0
       LayerScale-10             [-1, 257, 384]               0
        LayerNorm-11             [-1, 257, 384]             768
           Linear-12            [-1, 257, 1536]         591,360
             GELU-13            [-1, 257, 1536]               0
          Dropout-14            [-1, 25

MODEL SAVE

In [15]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict() if scheduler is not None else {},
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, optimizer= None, scheduler= None, metric='valid_dist'):

    checkpoint = torch.load(path, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    print("\nResuming training from epoch:", epoch)
    print('----------------------------------------\n')
    print("Epochs left: ", config['num_epochs'] - epoch)
    print("Optimizer: \n", optimizer)
    # print("Current Schedueler T_cur:", scheduler.T_cur)

    print("Best Val Dist:", metric)

    return [model, optimizer, scheduler, epoch, metric]

In [16]:
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=1e-6, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

WANDB

In [17]:
import wandb
# Uncomment to use wandb
if config['wandb']:
    wandb.login(key="1311b018830e3a64af503c0903514d9827af68eb")
    run = wandb.init(
        name=f"dinov2_{config['structure']}",
        entity="11789-dino-project-team-48",
        project='11785-DL-project',

        config=config,
        reinit=False
    )

    #Save model in a txt file
    model_arch  = str(model)
    arch_file   = open("model_arch.txt", "w")
    file_write  = arch_file.write(model_arch)
    arch_file.close()

    # Use wandb.Artifact to log the file
    artifact = wandb.Artifact('model_arch', type='model')
    artifact.add_file('model_arch.txt')
    wandb.log_artifact(artifact)

    wandb.watch(model, log="all")

INITIAL CHECKPOINTS

In [18]:
last_epoch_completed = 0
best_lev_dist = float("inf")

if RESUME_TRAINING:
    [model, optimizer, scheduler, epoch, metric] = load_model(PRE_CHECKPOINTS_PATH, model, optimizer, scheduler, metric='val_acc')

    last_epoch_completed = epoch
    best_lev_dist = metric


Resuming training from epoch: 26
----------------------------------------

Epochs left:  -6
Optimizer: 
 AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1.25e-07
    maximize: False
    weight_decay: 0.0001
)
Best Val Dist: 0.7547368421052632


In [19]:
checkpoint_best_model_filename = 'checkpoint-best-model.pth'
checkpoint_last_epoch_filename = 'checkpoint-last-epoch.pth'
epoch_model_path = os.path.join(CHECKPOINTS_ROOT, checkpoint_last_epoch_filename)
best_model_path = os.path.join(CHECKPOINTS_ROOT, checkpoint_best_model_filename)

Start Training

In [20]:
import gc
# Clear RAM for storage before you start training
torch.cuda.empty_cache()
gc.collect()

302

In [21]:
best_val_acc = 0.0

# Lists to store metrics for plotting
train_losses = []
val_losses = []
val_accs = []

for epoch in range(last_epoch_completed, config['num_epochs']):
    # Training
    model.train()
    train_loss = 0.0
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']} [Train]", leave=True)

    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_loop.set_postfix({'loss': f"{loss.item():.4f}"})

    train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{config['num_epochs']} [Val]", leave=True)

    with torch.no_grad():
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            val_loop.set_postfix({
                'loss': f"{loss.item():.4f}",
                'acc': f"{correct/total:.4f}"
            })

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    scheduler.step(val_loss)

    save_model(model, optimizer, scheduler, ['val_acc', val_acc], epoch, epoch_model_path)
    if config['wandb']:
        wandb.log({
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]['lr']
        })
        artifact_epoch = wandb.Artifact(f'epoch_model_{epoch}', type='model')
        artifact_epoch.add_file(epoch_model_path)
        wandb.log_artifact(artifact_epoch)
    print('Saving model...')

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save_model(model, optimizer, scheduler, ['val_acc', val_acc], epoch, best_model_path)
        if config['wandb']:
            artifact_best = wandb.Artifact('best_model', type='model')
            artifact_best.add_file(best_model_path)
            wandb.log_artifact(artifact_best)
        print(f"\nNew best model saved with val acc: {val_acc:.4f}")

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {optimizer.param_groups[0]['lr']}")

if config['wandb']:
    run.finish()

In [22]:
import matplotlib.pyplot as plt

if train_losses and val_losses and val_accs:
    # Plotting the metrics after training
    epochs = range(last_epoch_completed, config['num_epochs'])

    plt.figure(figsize=(12, 5))

    # Plot Train and Validation Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train & Validation Loss')
    plt.legend()

    # Plot Validation Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, val_accs, label='Val Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Validation Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [23]:
# --- Step 9: Final Evaluation ---
# model.load_state_dict(torch.load(best_model_path)['model_state_dict'])
model.eval()

test_correct = 0
test_total = 0
test_loop = tqdm(test_loader, desc='Testing', leave=True)

with torch.no_grad():
    for images, labels in test_loop:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        test_loop.set_postfix({'acc': f"{test_correct/test_total:.4f}"})

test_acc = test_correct / test_total
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

Testing: 100%|██████████| 60/60 [21:11<00:00, 21.20s/it, acc=0.7455]


Final Test Accuracy: 0.7455


Post Training Dynamic Quantization

In [28]:
class QuantizedDINOv2Classifier(nn.Module):
    def __init__(self, dinov2_model, quantized_classifier):
        super().__init__()
        self.dinov2 = dinov2_model
        self.classifier = quantized_classifier

    def forward(self, x):
        features = self.dinov2(x)
        features = self.dinov2.norm(features)
        return self.classifier(features)

quantized_dinov2 = torch.quantization.quantize_dynamic(
    model.dinov2, {nn.Linear}, dtype=torch.qint8
).to('cpu')

quantized_classifier = torch.quantization.quantize_dynamic(
    model.classifier, {nn.Linear}, dtype=torch.qint8
).to('cpu')

quantized_model = QuantizedDINOv2Classifier(quantized_dinov2, quantized_classifier).to("cpu")
summary(quantized_model, input_size=(3, 224, 224))

test_correct = 0
test_total = 0
test_loop = tqdm(test_loader, desc='Testing Quantized Model', leave=True)

with torch.no_grad():
    for images, labels in test_loop:
        images, labels = images.to('cpu'), labels.to('cpu')
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        test_loop.set_postfix({'acc': f"{test_correct/test_total:.4f}"})

test_acc = test_correct / test_total
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 384, 16, 16]         226,176
          Identity-2             [-1, 256, 384]               0
        PatchEmbed-3             [-1, 256, 384]               0
         LayerNorm-4             [-1, 257, 384]             768
            Linear-5            [-1, 257, 1152]               0
           Dropout-6          [-1, 6, 257, 257]               0
            Linear-7             [-1, 257, 384]               0
           Dropout-8             [-1, 257, 384]               0
   MemEffAttention-9             [-1, 257, 384]               0
       LayerScale-10             [-1, 257, 384]               0
        LayerNorm-11             [-1, 257, 384]             768
           Linear-12            [-1, 257, 1536]               0
             GELU-13            [-1, 257, 1536]               0
          Dropout-14            [-1, 25

Testing Quantized Model: 100%|██████████| 60/60 [19:17<00:00, 19.29s/it, acc=0.6953]


Final Test Accuracy: 0.6953


In [29]:
def count_params(model):
    return sum(p.numel() for p in model.parameters())

print("Total params for original dinov2:", count_params(model))
print("Total params for quantized dinov2:", count_params(quantized_model))

Total params for original dinov2: 22257800
Total params for quantized dinov2: 781440


In [33]:
from torchinfo import summary as info_summary
info_summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
DINOv2Classifier                              [1, 8]                    --
├─DinoVisionTransformer: 1-1                  [1, 384]                  526,848
│    └─PatchEmbed: 2-1                        [1, 256, 384]             --
│    │    └─Conv2d: 3-1                       [1, 384, 16, 16]          226,176
│    │    └─Identity: 3-2                     [1, 256, 384]             --
│    └─ModuleList: 2-2                        --                        --
│    │    └─NestedTensorBlock: 3-3            [1, 257, 384]             1,775,232
│    │    └─NestedTensorBlock: 3-4            [1, 257, 384]             1,775,232
│    │    └─NestedTensorBlock: 3-5            [1, 257, 384]             1,775,232
│    │    └─NestedTensorBlock: 3-6            [1, 257, 384]             1,775,232
│    │    └─NestedTensorBlock: 3-7            [1, 257, 384]             1,775,232
│    │    └─NestedTensorBlock: 3-8            [1, 

In [34]:
info_summary(quantized_model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                        Output Shape              Param #
QuantizedDINOv2Classifier                     [1, 8]                    --
├─DinoVisionTransformer: 1-1                  [1, 384]                  526,848
│    └─PatchEmbed: 2-1                        [1, 256, 384]             --
│    │    └─Conv2d: 3-1                       [1, 384, 16, 16]          226,176
│    │    └─Identity: 3-2                     [1, 256, 384]             --
│    └─ModuleList: 2-2                        --                        --
│    │    └─NestedTensorBlock: 3-3            [1, 257, 384]             2,304
│    │    └─NestedTensorBlock: 3-4            [1, 257, 384]             2,304
│    │    └─NestedTensorBlock: 3-5            [1, 257, 384]             2,304
│    │    └─NestedTensorBlock: 3-6            [1, 257, 384]             2,304
│    │    └─NestedTensorBlock: 3-7            [1, 257, 384]             2,304
│    │    └─NestedTensorBlock: 3-8            [1, 257, 384]           

Post Training Static Quantization